# Imports

In [95]:
import pandas as pd
import numpy as np
from sklearn.feature_selection import VarianceThreshold
from sklearn.cross_validation import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix, roc_auc_score, classification_report
import pprint as pp

# Read in the pandas.DataFrame from csv
data = pd.read_csv('../DataAnalysis/RandomForest/ebay_data_rf.csv', index_col=False)

# Random forest model

Our eBay data contains primarily categorical variables. Thus, random forest is a natural option for modeling the dataset, because the random forest algorithm is based on decision trees which naturally handle categorical data.

## Preprocessing

In [96]:
# Drop unwanted column(s)
data.drop('value', axis=1, inplace=True)

# Remove columns with zero variance
selector = VarianceThreshold()
selector.fit_transform(data)

# Separate target variable from inputs
y = data.sellingState
X = data.drop('sellingState', axis=1)

# Split test and train
test_frac = 0.25
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=test_frac, random_state=7
)

## Train the random forest classifier

In [97]:
n_estimators = 80
max_features = 7
weights = {0: 2, 1: 1}
clf = RandomForestClassifier(n_estimators = 80,
                             max_features=7,
                             class_weight=weights,
                             warm_start=False)

clf.fit(X_train,y_train)

RandomForestClassifier(bootstrap=True, class_weight={0: 2, 1: 1},
            criterion='gini', max_depth=None, max_features=7,
            max_leaf_nodes=None, min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=80, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

## Test the model on the test set

In [98]:
# Test on the training set:
y_test_pred = clf.predict(X_test)

# Print the confusion matrix
mat = confusion_matrix(y_test, y_test_pred)
num = np.sum(mat)
print("Confusion matrix:\n", mat)
print("Normalized confusion matrix:\n", mat/num)
print("Accuracy:\n", (mat[0,0]+mat[1,1])/num)

# Calculate the roc_auc score
print('ROC AUC:\n', roc_auc_score(y_test, clf.predict_proba(X_test)[:,1], average = 'weighted'))

Confusion matrix:
 [[1330  224]
 [ 323  623]]
Normalized confusion matrix:
 [[ 0.532   0.0896]
 [ 0.1292  0.2492]]
Accuracy:
 0.7812
ROC AUC:
 0.832348015488


The accuracy on the test set agrees well with the OOB error rate of about 0.2 for our model parameters.

## Feature importances
The random forest model also allows us to see which features were important for modeling the data.

In [100]:
cols = X.columns
feature_scores = clf.feature_importances_

score_card = pd.DataFrame.from_items([('Features', cols),('Scores', feature_scores)])
score_card.sort_values(by = 'Scores', inplace=True, ascending=False)

print(score_card)

                Features    Scores
17               endHour  0.131385
13             startHour  0.128735
19           endMonthday  0.110342
15         startMonthday  0.097369
6      buyItNowAvailable  0.091429
1        productId_value  0.073820
18            endWeekday  0.063566
14          startWeekday  0.059124
10          shippingType  0.043532
5            listingType  0.040778
9      expeditedShipping  0.028797
12       returnsAccepted  0.023537
3            conditionId  0.019814
7       bestOfferEnabled  0.018616
0         productId_type  0.018359
2   conditionDisplayName  0.018032
11        isShippingFree  0.010840
16            startMonth  0.009505
20              endMonth  0.005948
4                country  0.003612
8          paymentMethod  0.002860


We see that the most important features have to do with the time of day that the listing started and ended and the day of month that the listing started and ended. These variables are may be highly correlated, so we should be careful not to read too much into the model at this point. We can say, however, that listing time features are predictive for the listing outcome.